<img src="../assets/logos/Logo_layout.png" width=500></img>


# Analytics
Your first model just randomly places ships of size 2. Since you can make ships of many different shapes we would like to find what is the best shape. You also need to change the shape of your ships since other teams shooting models will learn the patterns of your layout. In this model, each game your model will choose one of two different layout strategies and you will then use [Amazon Athena](https://aws.amazon.com/athena/) to see the results. You can then repeat the experiment with new ship sizes and shapes to see if you can find better strategies. 

## Code 
The code for your second model is in /containers/Layout/2vs3 and contains the following files
    
- ### /containers/Layout/2vs3/Makefile
Makefile for creating the tar.gz file of code for [Amazon Sagemaker](https://aws.amazon.com/sagemaker/) framework containers

- ### [/containers/Layout/2vs3/host.py](/edit/GameDayRepo/containers/Layout/2vs3/host.py)
The code for generating inferences. In addition to randomly selecting a layout strategy it will also put a tag in the session attribute that we will use in athena to compare different games. You can change the tag value (which you should do for each model) by editing the host.py file. (don't run this file directly, use test.py)

- ### [/containers/Layout/2vs3/test.py](/edit/GameDayRepo/containers/Layout/2vs3/test.py)
code for testing the the host.py and train.py files locally. It creates a mock [Amazon Sagemaker](https://aws.amazon.com/sagemaker/) runtime inorder to run train.py and then test.py. You can edit this file inorder to customize your test. if you get a message about missing dependencies (like mxnet) just install them with:
```shell
pip install mxnet # for example
```

- ### [/containers/Layout/2vs3/train.py](/edit/GameDayRepo/containers/Layout/2vs3/train.py)
the training code (don't run this file directly, use test.py)


 Next, run the test script locally to confirm the scripts works. 
```shell
python ./test.py
```

## Configure
Before you can deploy your second model you need to configure the deployment. As described in the [Endpoint Notebook](../Endpoint_Reference.ipynb), you will edit to the deployment configuration in [/layout-config.json](/edit/GameDayRepo/layout-config.json). you need to edit the following field to the following values

|parameter field|Value|
|---|---|
|trainsourcefile|`layout-2vs3.tar.gz`|
|hostsourcefile|`layout-2vs3.tar.gz`|



## Deploy
1. follow instructions from endpoint reference to commit your code and changes and then push your changes to the remote branch
2. tell operations to deploy your changes

## Next Steps
Next, monitor you models performance. see [Athena](../Athena.ipynb) for details on setting up Athena (replace values between `<>` like in the Athena instructions). The following query will show win percentage by ship size (2 or 3). 
```sql
SELECT type,
         sum(won)/(sum(won)+sum(lost)) as won,
         sum(lost)/(sum(won)+sum(lost)) as lost
FROM 
    (SELECT json_extract(session,'$.size') AS size,
        CASE
            WHEN winner = '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS won,
        CASE
            WHEN winner != '<Team-Name>' THEN 1.0
            ELSE 0.0
        END AS lost
    FROM 
        (SELECT teama.session AS session,
         teama.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>"
        UNION
        SELECT teamb.session AS session,
         teamb.teamname AS team,
         winner
        FROM "<Game-data-catalog>"."<Game-data-bucket>" )
        WHERE team = '<Team-Name>' )
    GROUP BY  size
``` 
Now program some new ship sizes or shapes to do another round of experiments with, deploy it out, and rerun your athena query to see performance